In [1]:
import pandas as pd
import requests
import json
import csv
import os
import time

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
def get_acs_data(base_url, vars_str, fips_df, op_path, op_file, year, key, geo_type):
    
    """
    :param base_url: Different base url for 2000 and 2010 years
    :param fips_df: df with state and county fips codes
    :param op_file: op file name to which the data needs to be written
    :param census_year: year for which decennial census data
    :return:
    """
    
    # take a count variable to help distinguish 1st request from subsequent requests so that we take the variable
    # names only from the 1st call as the header names and skip varibale names from the subsequent calls.
    response = ''
    count = 0
    
    # Open a new file with the required output file name in append mode. 
    # To overwrite a file and write some new data into the file, we can open the file in the w mode, which will delete the old data from the file.
    with open(f'{op_path}/{op_file}', 'a') as file_wrtr:
        # take a csv writer object to write to this output file
        op_file_writer = csv.writer(file_wrtr)
    
        state_fips = fips_df['state'].unique().astype(str)
        # Iterate over each row in the state_county fips df
        for st_fips in state_fips:
            ## create the request url for each state and fips code to get the data for all county subdivisions under it ##
            try:
                """ Use timer if needed """
#                 if count in [20, 40, 60, 80, 100]:
#                     time.sleep(2) # milliseconds

                if st_fips.__len__() < 2:
                    st_fips = '0' + st_fips
                
                print('st_fips: ', st_fips)

                if geo_type == 'c_subd':
                    url = f'{base_url}/{year}/acs/acs5?get={vars_str}&for=county%20subdivision:*&in=state:{st_fips}%20county:*&key={key}'
                   # url = f'{base_url}/{year}/acs5?get={vars_str}&for=county%20subdivision:*&in=state:{st_fips}%20county:*'
                elif geo_type == 'place':
                    url = f'{base_url}/{year}/acs/acs5?get={vars_str}&for=place:*&in=state:{st_fips}&key={key}'
                elif geo_type =='county':
                    url = f'{base_url}/{year}/acs/acs5?get={vars_str}&for=county:*&in=state:{st_fips}&key={key}'

                print(f'url at count{count}: ', url)

                headers = requests.utils.default_headers()
                headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.90 Safari/537.36'
                response = requests.get(url, headers=headers)
                print('response: ', response)
                # load the json in to python object which would be list of all entries in the json object
                resp = json.loads(response.content)
                # If this is the first call(count=0), then write the entire content so that the 1st row with the variable names is the header
                if count == 0:
                    # iterate over respone python object and write each row to the csv
                    for res in resp:
                        op_file_writer.writerow(res)
                else:
                    # skipping 1st row which are the variable names starting from 2nd calls.
                    for res in resp[1:]:
                        op_file_writer.writerow(res)

                count += 1
                time.sleep(10)
                op_file_writer.writerow([])
                print('Total API Calls: ', count)
            except Exception as ex:
                print('Error response', response)
                print('Exception: ', ex)
                break

### GET EDUC SCHL DATA

In [75]:
years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]


st_cnty_fips_10 = pd.read_csv('/Users/salma/Research/dissertation_work/data/educ_schl_acs/st_cnty_fips_2010.csv')

# get county subdivision level data
for year in years:
    print('year: ', year)
    get_acs_data(base_url='https://api.census.gov/data', 
                             vars_str='NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003E,B15001_011E,B15001_043E,B15001_044E,B15001_052E,B15002_001E,B15002_002E,B15002_019E,C15002A_001E,C15002A_002E,C15002A_007E,C15002B_001E,C15002B_002E,C15002B_007E,C15002H_001E,C15002H_002E,C15002H_007E,C15002I_001E,C15002I_002E,C15002I_007E', 
                             fips_df=st_cnty_fips_10,
                             key='99d3a2679609fec77eb6c7de82884928a52310d4',
                             geo_type='county',
                             year = year,
                             op_path='/Users/salma/Research/dissertation_work/data/educ_schl_acs/cnty',
                             op_file=f'educ_school_acs_initial_{year}.csv')

year:  2013
st_fips:  01
url at count0:  https://api.census.gov/data/2013/acs/acs5?get=NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003E,B15001_011E,B15001_043E,B15001_044E,B15001_052E,B15002_001E,B15002_002E,B15002_019E,C15002A_001E,C15002A_002E,C15002A_007E,C15002B_001E,C15002B_002E,C15002B_007E,C15002H_001E,C15002H_002E,C15002H_007E,C15002I_001E,C15002I_002E,C15002I_007E&for=county:*&in=state:01&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  1
st_fips:  02
url at count1:  https://api.census.gov/data/2013/acs/acs5?get=NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003E,B15001_011E,B15001_043E,B15001_044E,B1

response:  <Response [200]>
Total API Calls:  13
st_fips:  17
url at count13:  https://api.census.gov/data/2013/acs/acs5?get=NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003E,B15001_011E,B15001_043E,B15001_044E,B15001_052E,B15002_001E,B15002_002E,B15002_019E,C15002A_001E,C15002A_002E,C15002A_007E,C15002B_001E,C15002B_002E,C15002B_007E,C15002H_001E,C15002H_002E,C15002H_007E,C15002I_001E,C15002I_002E,C15002I_007E&for=county:*&in=state:17&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  14
st_fips:  18
url at count14:  https://api.census.gov/data/2013/acs/acs5?get=NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003

response:  <Response [200]>
Total API Calls:  26
st_fips:  30
url at count26:  https://api.census.gov/data/2013/acs/acs5?get=NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003E,B15001_011E,B15001_043E,B15001_044E,B15001_052E,B15002_001E,B15002_002E,B15002_019E,C15002A_001E,C15002A_002E,C15002A_007E,C15002B_001E,C15002B_002E,C15002B_007E,C15002H_001E,C15002H_002E,C15002H_007E,C15002I_001E,C15002I_002E,C15002I_007E&for=county:*&in=state:30&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  27
st_fips:  31
url at count27:  https://api.census.gov/data/2013/acs/acs5?get=NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003

response:  <Response [200]>
Total API Calls:  1
st_fips:  02
url at count1:  https://api.census.gov/data/2017/acs/acs5?get=NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003E,B15001_011E,B15001_043E,B15001_044E,B15001_052E,B15002_001E,B15002_002E,B15002_019E,C15002A_001E,C15002A_002E,C15002A_007E,C15002B_001E,C15002B_002E,C15002B_007E,C15002H_001E,C15002H_002E,C15002H_007E,C15002I_001E,C15002I_002E,C15002I_007E&for=county:*&in=state:02&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  2
st_fips:  04
url at count2:  https://api.census.gov/data/2017/acs/acs5?get=NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003E,B1

response:  <Response [200]>
Total API Calls:  14
st_fips:  18
url at count14:  https://api.census.gov/data/2017/acs/acs5?get=NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003E,B15001_011E,B15001_043E,B15001_044E,B15001_052E,B15002_001E,B15002_002E,B15002_019E,C15002A_001E,C15002A_002E,C15002A_007E,C15002B_001E,C15002B_002E,C15002B_007E,C15002H_001E,C15002H_002E,C15002H_007E,C15002I_001E,C15002I_002E,C15002I_007E&for=county:*&in=state:18&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  15
st_fips:  19
url at count15:  https://api.census.gov/data/2017/acs/acs5?get=NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003

### Timing out after random calls - so trying to see if we can get data for 1 year from the last successful st_fips

In [ ]:
st_cnty_fips_10 = pd.read_csv('/Users/salma/Research/dissertation_work/data/educ_schl_acs/st_fips_10_test.csv')

get_educ_school_acs_data(base_url='https://api.census.gov/data', 
                             vars_str='NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003E,B15001_011E,B15001_043E,B15001_044E,B15001_052E,B15002_001E,B15002_002E,B15002_019E,C15002A_001E,C15002A_002E,C15002A_007E,C15002B_001E,C15002B_002E,C15002B_007E,C15002H_001E,C15002H_002E,C15002H_007E,C15002I_001E,C15002I_002E,C15002I_007E',
                             fips_df=st_cnty_fips_10,
                             key='99d3a2679609fec77eb6c7de82884928a52310d4',
                             geo_type='county',
                             year = 2020,
                             op_path='/Users/salma/Research/dissertation_work/data/educ_schl_acs/cnty',
                             op_file=f'educ_school_acs_initial_2020.csv')

### GET ECON DATA

In [12]:
# years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

years = [2015, 2020]

st_cnty_fips_10 = pd.read_csv('/Users/salma/Research/dissertation_work/data/st_cnty_fips_2010.csv')

for year in years:
    get_acs_data(base_url='https://api.census.gov/data', 
                             vars_str='NAME,B23001_001E,B23001_002E,B23001_088E,C23002B_001E,C23002B_002E,C23002B_015E,C23002H_001E,C23002H_002E,C23002H_015E,C23002I_001E,C23002I_002E,C23002I_015E,B19301_001E,B19301B_001E,B19301H_001E,B19301I_001E', 
                             fips_df=st_cnty_fips_10,
                             key='99d3a2679609fec77eb6c7de82884928a52310d4',
                             geo_type='place',
                             year = year,
                             op_path='/Users/salma/Research/dissertation_work/data/economic/place',
                             op_file=f'econ_ini_place_{year}.csv')

st_fips:  01
url at count0:  https://api.census.gov/data/2015/acs/acs5?get=NAME,B23001_001E,B23001_002E,B23001_088E,C23002B_001E,C23002B_002E,C23002B_015E,C23002H_001E,C23002H_002E,C23002H_015E,C23002I_001E,C23002I_002E,C23002I_015E,B19301_001E,B19301B_001E,B19301H_001E,B19301I_001E&for=place:*&in=state:01&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  1
st_fips:  02
url at count1:  https://api.census.gov/data/2015/acs/acs5?get=NAME,B23001_001E,B23001_002E,B23001_088E,C23002B_001E,C23002B_002E,C23002B_015E,C23002H_001E,C23002H_002E,C23002H_015E,C23002I_001E,C23002I_002E,C23002I_015E,B19301_001E,B19301B_001E,B19301H_001E,B19301I_001E&for=place:*&in=state:02&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  2
st_fips:  04
url at count2:  https://api.census.gov/data/2015/acs/acs5?get=NAME,B23001_001E,B23001_002E,B23001_088E,C23002B_001E,C23002B_002E,C23002B_015E,C23002H_001E,C23002H_002E,C23002H_015E,C230

response:  <Response [200]>
Total API Calls:  21
st_fips:  25
url at count21:  https://api.census.gov/data/2015/acs/acs5?get=NAME,B23001_001E,B23001_002E,B23001_088E,C23002B_001E,C23002B_002E,C23002B_015E,C23002H_001E,C23002H_002E,C23002H_015E,C23002I_001E,C23002I_002E,C23002I_015E,B19301_001E,B19301B_001E,B19301H_001E,B19301I_001E&for=place:*&in=state:25&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  22
st_fips:  26
url at count22:  https://api.census.gov/data/2015/acs/acs5?get=NAME,B23001_001E,B23001_002E,B23001_088E,C23002B_001E,C23002B_002E,C23002B_015E,C23002H_001E,C23002H_002E,C23002H_015E,C23002I_001E,C23002I_002E,C23002I_015E,B19301_001E,B19301B_001E,B19301H_001E,B19301I_001E&for=place:*&in=state:26&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  23
st_fips:  27
url at count23:  https://api.census.gov/data/2015/acs/acs5?get=NAME,B23001_001E,B23001_002E,B23001_088E,C23002B_001E,C23002B_002E,C2

response:  <Response [200]>
Total API Calls:  42
st_fips:  47
url at count42:  https://api.census.gov/data/2015/acs/acs5?get=NAME,B23001_001E,B23001_002E,B23001_088E,C23002B_001E,C23002B_002E,C23002B_015E,C23002H_001E,C23002H_002E,C23002H_015E,C23002I_001E,C23002I_002E,C23002I_015E,B19301_001E,B19301B_001E,B19301H_001E,B19301I_001E&for=place:*&in=state:47&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  43
st_fips:  48
url at count43:  https://api.census.gov/data/2015/acs/acs5?get=NAME,B23001_001E,B23001_002E,B23001_088E,C23002B_001E,C23002B_002E,C23002B_015E,C23002H_001E,C23002H_002E,C23002H_015E,C23002I_001E,C23002I_002E,C23002I_015E,B19301_001E,B19301B_001E,B19301H_001E,B19301I_001E&for=place:*&in=state:48&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  44
st_fips:  49
url at count44:  https://api.census.gov/data/2015/acs/acs5?get=NAME,B23001_001E,B23001_002E,B23001_088E,C23002B_001E,C23002B_002E,C2

response:  <Response [200]>
Total API Calls:  12
st_fips:  16
url at count12:  https://api.census.gov/data/2020/acs/acs5?get=NAME,B23001_001E,B23001_002E,B23001_088E,C23002B_001E,C23002B_002E,C23002B_015E,C23002H_001E,C23002H_002E,C23002H_015E,C23002I_001E,C23002I_002E,C23002I_015E,B19301_001E,B19301B_001E,B19301H_001E,B19301I_001E&for=place:*&in=state:16&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  13
st_fips:  17
url at count13:  https://api.census.gov/data/2020/acs/acs5?get=NAME,B23001_001E,B23001_002E,B23001_088E,C23002B_001E,C23002B_002E,C23002B_015E,C23002H_001E,C23002H_002E,C23002H_015E,C23002I_001E,C23002I_002E,C23002I_015E,B19301_001E,B19301B_001E,B19301H_001E,B19301I_001E&for=place:*&in=state:17&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  14
st_fips:  18
url at count14:  https://api.census.gov/data/2020/acs/acs5?get=NAME,B23001_001E,B23001_002E,B23001_088E,C23002B_001E,C23002B_002E,C2

response:  <Response [200]>
Total API Calls:  33
st_fips:  37
url at count33:  https://api.census.gov/data/2020/acs/acs5?get=NAME,B23001_001E,B23001_002E,B23001_088E,C23002B_001E,C23002B_002E,C23002B_015E,C23002H_001E,C23002H_002E,C23002H_015E,C23002I_001E,C23002I_002E,C23002I_015E,B19301_001E,B19301B_001E,B19301H_001E,B19301I_001E&for=place:*&in=state:37&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  34
st_fips:  38
url at count34:  https://api.census.gov/data/2020/acs/acs5?get=NAME,B23001_001E,B23001_002E,B23001_088E,C23002B_001E,C23002B_002E,C23002B_015E,C23002H_001E,C23002H_002E,C23002H_015E,C23002I_001E,C23002I_002E,C23002I_015E,B19301_001E,B19301B_001E,B19301H_001E,B19301I_001E&for=place:*&in=state:38&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  35
st_fips:  39
url at count35:  https://api.census.gov/data/2020/acs/acs5?get=NAME,B23001_001E,B23001_002E,B23001_088E,C23002B_001E,C23002B_002E,C2

### Timing out after random calls - so trying to see if we can get data for 1 year from the last successful st_fips

In [11]:
st_cnty_fips_10 = pd.read_csv('/Users/salma/Research/dissertation_work/data/st_cnty_fips_2010.csv')

get_acs_data(base_url='https://api.census.gov/data', 
                             vars_str='NAME,B23001_001E,B23001_002E,B23001_088E,C23002B_001E,C23002B_002E,C23002B_015E,C23002H_001E,C23002H_002E,C23002H_015E,C23002I_001E,C23002I_002E,C23002I_015E,B19301_001E,B19301B_001E,B19301H_001E,B19301I_001E', 
                             fips_df=st_cnty_fips_10,
                             key='99d3a2679609fec77eb6c7de82884928a52310d4',
                             geo_type='place',
                             year = 2015,
                             op_path='/Users/salma/Research/dissertation_work/data/economic/place',
                             op_file=f'econ_ini_place_{year}.csv')

st_fips:  01
url at count0:  https://api.census.gov/data/2015/acs/acs5?get=NAME,B23001_001E,B23001_002E,B23001_088E,C23002B_001E,C23002B_002E,C23002B_015E,C23002H_001E,C23002H_002E,C23002H_015E,C23002I_001E,C23002I_002E,C23002I_015E,B19301_001E,B19301B_001E,B19301H_001E,B19301I_001E&for=place:*&in=state:01&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  1
st_fips:  02
url at count1:  https://api.census.gov/data/2015/acs/acs5?get=NAME,B23001_001E,B23001_002E,B23001_088E,C23002B_001E,C23002B_002E,C23002B_015E,C23002H_001E,C23002H_002E,C23002H_015E,C23002I_001E,C23002I_002E,C23002I_015E,B19301_001E,B19301B_001E,B19301H_001E,B19301I_001E&for=place:*&in=state:02&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  2
st_fips:  04
url at count2:  https://api.census.gov/data/2015/acs/acs5?get=NAME,B23001_001E,B23001_002E,B23001_088E,C23002B_001E,C23002B_002E,C23002B_015E,C23002H_001E,C23002H_002E,C23002H_015E,C230

KeyboardInterrupt: 

### GET DEMO DATA

In [ ]:
POP100
White_count
Black_count
Hispanic_count
Age1524_WhiteM
White_Males_All
Age1524_WhiteF
White_Females_All
Age1524_BlackM
Black_Males_All
Age1524_BlackF
Black_Females_All
Hispanic_Males_All
Age1524_HispanicM
Age1524_HispanicF
Hispanic_Females_All
Pct_WYM
Pct_WYF
total_count_county
population
population_mean

In [15]:
years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

st_cnty_fips_10 = pd.read_csv('/Users/salma/Research/dissertation_work/data/st_cnty_fips_2010.csv')

for year in years:
    get_acs_data(base_url='https://api.census.gov/data', 
                             vars_str='NAME,B01001_001E,B01001_002E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_026E,B01001_030E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001B_001E,B01001B_002E,B01001B_006E,B01001B_007E,B01001B_008E,B01001B_017E,B01001B_021E,B01001B_022E,B01001B_023E,B01001H_001E,B01001H_002E,B01001H_006E,B01001H_007E,B01001H_008E,B01001H_017E,B01001H_021E,B01001H_022E,B01001H_023E,B01001I_001E,B01001I_002E,B01001I_006E,B01001I_007E,B01001I_008E,B01001I_017E,B01001I_021E,B01001I_022E,B01001I_023E',
                             fips_df=st_cnty_fips_10,
                             key='99d3a2679609fec77eb6c7de82884928a52310d4',
                             geo_type='place',
                             year = year,
                             op_path='/Users/salma/Research/dissertation_work/data/demo/place',
                             op_file=f'demo_ini_place_{year}.csv')

st_fips:  01
url at count0:  https://api.census.gov/data/2010/acs/acs5?get=NAME,B01001_001E,B01001_002E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_026E,B01001_030E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001B_001E,B01001B_002E,B01001B_006E,B01001B_007E,B01001B_008E,B01001B_017E,B01001B_021E,B01001B_022E,B01001B_023E,B01001H_001E,B01001H_002E,B01001H_006E,B01001H_007E,B01001H_008E,B01001H_017E,B01001H_021E,B01001H_022E,B01001H_023E,B01001I_001E,B01001I_002E,B01001I_006E,B01001I_007E,B01001I_008E,B01001I_017E,B01001I_021E,B01001I_022E,B01001I_023E&for=place:*&in=state:01&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  1
st_fips:  02
url at count1:  https://api.census.gov/data/2010/acs/acs5?get=NAME,B01001_001E,B01001_002E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_026E,B01001_030E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001B_001E

response:  <Response [200]>
Total API Calls:  12
st_fips:  16
url at count12:  https://api.census.gov/data/2010/acs/acs5?get=NAME,B01001_001E,B01001_002E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_026E,B01001_030E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001B_001E,B01001B_002E,B01001B_006E,B01001B_007E,B01001B_008E,B01001B_017E,B01001B_021E,B01001B_022E,B01001B_023E,B01001H_001E,B01001H_002E,B01001H_006E,B01001H_007E,B01001H_008E,B01001H_017E,B01001H_021E,B01001H_022E,B01001H_023E,B01001I_001E,B01001I_002E,B01001I_006E,B01001I_007E,B01001I_008E,B01001I_017E,B01001I_021E,B01001I_022E,B01001I_023E&for=place:*&in=state:16&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  13
st_fips:  17
url at count13:  https://api.census.gov/data/2010/acs/acs5?get=NAME,B01001_001E,B01001_002E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_026E,B01001_030E,B01001_0

response:  <Response [200]>
Total API Calls:  24
st_fips:  28
url at count24:  https://api.census.gov/data/2010/acs/acs5?get=NAME,B01001_001E,B01001_002E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_026E,B01001_030E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001B_001E,B01001B_002E,B01001B_006E,B01001B_007E,B01001B_008E,B01001B_017E,B01001B_021E,B01001B_022E,B01001B_023E,B01001H_001E,B01001H_002E,B01001H_006E,B01001H_007E,B01001H_008E,B01001H_017E,B01001H_021E,B01001H_022E,B01001H_023E,B01001I_001E,B01001I_002E,B01001I_006E,B01001I_007E,B01001I_008E,B01001I_017E,B01001I_021E,B01001I_022E,B01001I_023E&for=place:*&in=state:28&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  25
st_fips:  29
url at count25:  https://api.census.gov/data/2010/acs/acs5?get=NAME,B01001_001E,B01001_002E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_026E,B01001_030E,B01001_0

response:  <Response [200]>
Total API Calls:  36
st_fips:  40
url at count36:  https://api.census.gov/data/2010/acs/acs5?get=NAME,B01001_001E,B01001_002E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_026E,B01001_030E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001B_001E,B01001B_002E,B01001B_006E,B01001B_007E,B01001B_008E,B01001B_017E,B01001B_021E,B01001B_022E,B01001B_023E,B01001H_001E,B01001H_002E,B01001H_006E,B01001H_007E,B01001H_008E,B01001H_017E,B01001H_021E,B01001H_022E,B01001H_023E,B01001I_001E,B01001I_002E,B01001I_006E,B01001I_007E,B01001I_008E,B01001I_017E,B01001I_021E,B01001I_022E,B01001I_023E&for=place:*&in=state:40&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  37
st_fips:  41
url at count37:  https://api.census.gov/data/2010/acs/acs5?get=NAME,B01001_001E,B01001_002E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_026E,B01001_030E,B01001_0

response:  <Response [200]>
Total API Calls:  48
st_fips:  54
url at count48:  https://api.census.gov/data/2010/acs/acs5?get=NAME,B01001_001E,B01001_002E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_026E,B01001_030E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001B_001E,B01001B_002E,B01001B_006E,B01001B_007E,B01001B_008E,B01001B_017E,B01001B_021E,B01001B_022E,B01001B_023E,B01001H_001E,B01001H_002E,B01001H_006E,B01001H_007E,B01001H_008E,B01001H_017E,B01001H_021E,B01001H_022E,B01001H_023E,B01001I_001E,B01001I_002E,B01001I_006E,B01001I_007E,B01001I_008E,B01001I_017E,B01001I_021E,B01001I_022E,B01001I_023E&for=place:*&in=state:54&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  49
st_fips:  55
url at count49:  https://api.census.gov/data/2010/acs/acs5?get=NAME,B01001_001E,B01001_002E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_026E,B01001_030E,B01001_0

...

response:  <Response [200]>
Total API Calls:  45
st_fips:  50
url at count45:  https://api.census.gov/data/2019/acs/acs5?get=NAME,B01001_001E,B01001_002E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_026E,B01001_030E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001B_001E,B01001B_002E,B01001B_006E,B01001B_007E,B01001B_008E,B01001B_017E,B01001B_021E,B01001B_022E,B01001B_023E,B01001H_001E,B01001H_002E,B01001H_006E,B01001H_007E,B01001H_008E,B01001H_017E,B01001H_021E,B01001H_022E,B01001H_023E,B01001I_001E,B01001I_002E,B01001I_006E,B01001I_007E,B01001I_008E,B01001I_017E,B01001I_021E,B01001I_022E,B01001I_023E&for=place:*&in=state:50&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  46
st_fips:  51
url at count46:  https://api.census.gov/data/2019/acs/acs5?get=NAME,B01001_001E,B01001_002E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_026E,B01001_030E,B01001_0

response:  <Response [200]>
Total API Calls:  6
st_fips:  09
url at count6:  https://api.census.gov/data/2020/acs/acs5?get=NAME,B01001_001E,B01001_002E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_026E,B01001_030E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001B_001E,B01001B_002E,B01001B_006E,B01001B_007E,B01001B_008E,B01001B_017E,B01001B_021E,B01001B_022E,B01001B_023E,B01001H_001E,B01001H_002E,B01001H_006E,B01001H_007E,B01001H_008E,B01001H_017E,B01001H_021E,B01001H_022E,B01001H_023E,B01001I_001E,B01001I_002E,B01001I_006E,B01001I_007E,B01001I_008E,B01001I_017E,B01001I_021E,B01001I_022E,B01001I_023E&for=place:*&in=state:09&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  7
st_fips:  10
url at count7:  https://api.census.gov/data/2020/acs/acs5?get=NAME,B01001_001E,B01001_002E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_026E,B01001_030E,B01001_031E,

response:  <Response [200]>
Total API Calls:  18
st_fips:  22
url at count18:  https://api.census.gov/data/2020/acs/acs5?get=NAME,B01001_001E,B01001_002E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_026E,B01001_030E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001B_001E,B01001B_002E,B01001B_006E,B01001B_007E,B01001B_008E,B01001B_017E,B01001B_021E,B01001B_022E,B01001B_023E,B01001H_001E,B01001H_002E,B01001H_006E,B01001H_007E,B01001H_008E,B01001H_017E,B01001H_021E,B01001H_022E,B01001H_023E,B01001I_001E,B01001I_002E,B01001I_006E,B01001I_007E,B01001I_008E,B01001I_017E,B01001I_021E,B01001I_022E,B01001I_023E&for=place:*&in=state:22&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  19
st_fips:  23
url at count19:  https://api.census.gov/data/2020/acs/acs5?get=NAME,B01001_001E,B01001_002E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_026E,B01001_030E,B01001_0

response:  <Response [200]>
Total API Calls:  30
st_fips:  34
url at count30:  https://api.census.gov/data/2020/acs/acs5?get=NAME,B01001_001E,B01001_002E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_026E,B01001_030E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001B_001E,B01001B_002E,B01001B_006E,B01001B_007E,B01001B_008E,B01001B_017E,B01001B_021E,B01001B_022E,B01001B_023E,B01001H_001E,B01001H_002E,B01001H_006E,B01001H_007E,B01001H_008E,B01001H_017E,B01001H_021E,B01001H_022E,B01001H_023E,B01001I_001E,B01001I_002E,B01001I_006E,B01001I_007E,B01001I_008E,B01001I_017E,B01001I_021E,B01001I_022E,B01001I_023E&for=place:*&in=state:34&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  31
st_fips:  35
url at count31:  https://api.census.gov/data/2020/acs/acs5?get=NAME,B01001_001E,B01001_002E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_026E,B01001_030E,B01001_0

response:  <Response [200]>
Total API Calls:  42
st_fips:  47
url at count42:  https://api.census.gov/data/2020/acs/acs5?get=NAME,B01001_001E,B01001_002E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_026E,B01001_030E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001B_001E,B01001B_002E,B01001B_006E,B01001B_007E,B01001B_008E,B01001B_017E,B01001B_021E,B01001B_022E,B01001B_023E,B01001H_001E,B01001H_002E,B01001H_006E,B01001H_007E,B01001H_008E,B01001H_017E,B01001H_021E,B01001H_022E,B01001H_023E,B01001I_001E,B01001I_002E,B01001I_006E,B01001I_007E,B01001I_008E,B01001I_017E,B01001I_021E,B01001I_022E,B01001I_023E&for=place:*&in=state:47&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  43
st_fips:  48
url at count43:  https://api.census.gov/data/2020/acs/acs5?get=NAME,B01001_001E,B01001_002E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_026E,B01001_030E,B01001_0